In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import pathlib
from skimage.measure import regionprops, label
from pathlib import Path
import torch
import cv2
import edt
import matplotlib as mpl
from math import floor
mpl.use('QtAgg')
%matplotlib qt5

In [2]:
filepath = Path('/home/pk/Documents/rtseg/data/backbone/000_img_masks.tif')

In [3]:
image = imread(filepath)

In [4]:
plt.figure()
plt.imshow(image)
plt.show()

In [5]:
np.unique(image)

array([  0, 448, 462, 489, 503, 510, 513, 528, 548, 554, 557, 570, 579,
       590, 603, 619, 631], dtype=uint16)

#### __Compute the backbone of each region on the image__

#### Fixed parameters

In [6]:
props = regionprops(image)

In [7]:
props[0].bbox, props[0].label

((10, 16, 41, 36), 448)

In [8]:
plt.figure()
plt.imshow(props[0].image)
plt.show()

In [9]:
dilation_threshold = 3
dilation_radius = floor(dilation_threshold/2)

In [10]:
dilation_radius

1

In [11]:
img = props[0].image

In [12]:
padded_img = np.pad(img, (dilation_radius+1, dilation_radius+1)).astype('uint8')

In [13]:
img.shape, padded_img.shape

((31, 20), (35, 24))

In [14]:
se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilation_radius+2, dilation_radius+2))

In [15]:
se

array([[0, 1, 0],
       [1, 1, 1],
       [0, 1, 0]], dtype=uint8)

In [16]:
dilated_img = cv2.dilate(padded_img, se)

In [17]:
dilated_img.shape

(35, 24)

In [18]:
xor = np.logical_xor(padded_img, dilated_img)

In [19]:
plt.imshow(xor)

In [20]:
plt.imshow(dilated_img)

In [20]:
region_edt = edt.edt(dilated_img)

In [21]:
plt.imshow(region_edt)

In [22]:
blob_boundary = (region_edt == 1)


In [128]:
label_dist = region_edt * (region_edt >= dilation_threshold)

In [129]:
final_labelled_image = label_dist[dilation_radius+1: -dilation_radius-1, dilation_radius+1: -dilation_radius-1]

In [130]:
final_labelled_image.shape

(31, 20)

In [131]:
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(props[0].image)
ax[1].imshow(final_labelled_image)
plt.show()

In [132]:
y_coords, x_coords = np.nonzero(final_labelled_image)

In [133]:
pixel_values = final_labelled_image[y_coords, x_coords]

In [134]:
y_coords.shape, x_coords.shape, pixel_values.shape

((190,), (190,), (190,))

#### I am changing this cubing to a parameter, as I notice some of them are not great

In [135]:
weights = np.sqrt(pixel_values ** 3)

In [136]:
weights.shape

(190,)

In [137]:
A = np.vstack((x_coords**2, x_coords, np.ones(x_coords.shape)))

In [138]:
A.shape

(3, 190)

In [139]:
w_rep = np.tile(weights, [3, 1])

In [140]:
wA = (w_rep * A).T
wB = weights * y_coords

In [141]:
wA.shape, wB.shape

((190, 3), (190,))

In [142]:
from numpy.linalg import lstsq

In [143]:
fit_coeff,_, _, _ = lstsq(wA, wB, rcond=-1)

In [144]:
fit_coeff

array([ 0.01549774,  1.60419873, -3.85620333])

In [145]:
plt.figure()
plt.imshow(final_labelled_image)
x = np.linspace(0.5, final_labelled_image.shape[1]-0.5, num=100)
y = fit_coeff[0] * x**2 + fit_coeff[1] * x + fit_coeff[2]
plt.plot(x, y, 'r-')
plt.show()

#### Compute backbone pixels

In [146]:
img.shape

(31, 20)

In [147]:
img_size = img.shape

In [148]:
x_data = np.arange(-0.5, img_size[1]+0.5)
y_data = fit_coeff[0] * x_data**2 + fit_coeff[1] * x_data + fit_coeff[2]

In [149]:
plt.figure()
plt.imshow(img)
plt.plot(x_data, y_data, 'r--')
plt.show()

##### There are cases. one for a straight line and the other for a quadratic curve

In [153]:
round(y_data[0])

-5

In [154]:
y_data[-1]

33.31868872517943

In [164]:
if abs(fit_coeff[0]) < 1e-4 and round(y_data[0]) == round(y_data[-1]):
    fit_coord = np.vstack((np.arange(0, img_size[1]), round(y_data[0]) * np.ones((img_size[1],)))).T
else:
    

In [175]:
y_data_round = np.round(y_data)
stat_pt = -fit_coeff[1] / (2 * fit_coeff[0]) #  ax^2 + bx + c = 0 --> -b/2a is the point where derivative==0
if stat_pt > -0.5 && stat_pt < img_size[1] - 1:
    round_stat_pt_val = round(fit_coeff[0] * (stat_pt**2) + fit_coeff[1] * stat_pt + fit_coeff[2])
    round_stat_pt = round(stat_pt)
    upp_bound = np.zeros((img_size[1]),) # upper bound defined for each pixel on the x-axis
    low_bound = np.zeros((img_size[1]),) # lower bound defined for each pixel on the x-axis

    if fit_coeff[0] > 0:
        upp_bound[0:round_stat_pt] = y_data_round[0:round_stat_pt]
        upp_bound[round_stat_pt] = max(y_data_round[round_stat_pt:round_stat_pt+2])
        upp_bound[round_stat_pt+1:] = y_data_round[round_stat_pt+1:];

        low_bound[0:round_stat_pt] = y_data_round[1:round_stat_pt+1]
        round_down_point = 2
        low_bound[] = 
        low_bound[] = 
        low_bound[] = 
                                       
    else:
        upp_bound[] = 0
        upp_bound[] = 0
        upp_bound[] = 0
        low_bound[] = 0
        low_bound[] = 0
        low_bound[] = 0
else:
    if (fit_coeff[0] > 0 and stat_pt > img_size[1]-1) or (fit_coeff[0] < 0 and stat_pt < -0.5):
        upp_bound = 
        low_bound = 
        round_down_point
        low_bound
    else:
        upp_bound = y_data_round[1:]
        low_bound = y_data_round[:-1]
    

# clip the upper bound and lower bounds
# and populate the pixels

SyntaxError: invalid syntax (3647062420.py, line 3)

In [176]:
a = np.arange(10)

In [177]:
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [179]:
a[:-1]

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [172]:
stat_pt

-51.75588214590985

(20,)

In [ ]:
def compute_backbone_poles():
    """
    For each cell in the image, when you call regionprops, you also want to 
    calculate the backbones and poles of each cell in the image.
    This function is meant to be in the extra_properties arguement in 
    skimage.measure.regionprops fuction.

    This function will add extra properties to each region in the 
    regionprops list.

    The extra properties are 
        1. fit_coeff: [a, b, c] 
        2. poles: 
        3. arc_length: 
        4. backbone_coordinates:

    Backbone is fit using a quadratic equation.

    
    Args:

    Returns:
    
    """
    pass
def compute_arc_length():
    """
    Computes the arc length
    """
    pass

#### __Computing internal coordinate of the dot__

In [ ]:
def get_projected_point():
    """
    
    """
    pass

In [16]:
def compute_internal_coordinate():
    """
    """
    pass